In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.io import *
from fastai.conv_learner import *

from fastai.column_data import *

## Setup

We're going to download the collected works of Nietzsche to use as our data for this class.

In [2]:
PATH='data/nietzsche/'

In [3]:
text = open(f'{PATH}nietzschesmall.txt').read()
print('corpus length:', len(text))

corpus length: 40000


In [4]:
text[:400]

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand women--that the terrible\nseriousness and clumsy importunity with which they have usually paid\ntheir addresses to Truth, have been unskilled and unseemly methods for\nwinning a woman? Certainly she has never allowed herself '

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 76


Sometimes it's useful to have a zero value in the dataset, e.g. for padding

In [6]:
chars.insert(0, "\0")

''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXY_abcdefghijklmnopqrst'

Map from chars to indices and back again

In [7]:
char_indices = {c: i for i, c in enumerate(chars)}
indices_char = {i: c for i, c in enumerate(chars)}

*idx* will be the data we use from now on - it simply converts all the characters to their index (based on the mapping above)

In [8]:
idx = [char_indices[c] for c in text]

idx[:10]

[39, 41, 28, 29, 24, 26, 28, 1, 1, 1]

In [9]:
''.join(indices_char[i] for i in idx[:70])

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not gro'

## Three char model

### Create inputs

Create a list of every 4th character, starting at the 0th, 1st, 2nd, then 3rd characters

In [14]:
cs=3
c1_dat = [idx[i]   for i in range(0, len(idx)-cs, cs)]
c2_dat = [idx[i+1] for i in range(0, len(idx)-cs, cs)]
c3_dat = [idx[i+2] for i in range(0, len(idx)-cs, cs)]
c4_dat = [idx[i+3] for i in range(0, len(idx)-cs, cs)]

Our inputs

In [15]:
x1 = np.stack(c1_dat)
x2 = np.stack(c2_dat)
x3 = np.stack(c3_dat)

Our output

In [16]:
y = np.stack(c4_dat)

The first 4 inputs and outputs

In [17]:
x1[:4], x2[:4], x3[:4]

(array([39, 29, 28,  1]), array([41, 24,  1, 42]), array([28, 26,  1, 44]))

In [18]:
y[:4]

array([29, 28,  1, 39])

In [19]:
x1.shape, y.shape

((13333,), (13333,))

### Create and train model

Pick a size for our hidden state

In [20]:
n_hidden = 256

The number of latent factors to create (i.e. the size of the embedding matrix)

In [21]:
n_fac = 42

In [25]:
class Char3Model(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)

        # The 'green arrow' from our diagram - the layer operation from input to hidden
        self.l_in = nn.Linear(n_fac, n_hidden)

        # The 'orange arrow' from our diagram - the layer operation from hidden to hidden
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        
        # The 'blue arrow' from our diagram - the layer operation from hidden to output
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, c1, c2, c3):
        in1 = F.relu(self.l_in(self.e(c1)))
        in2 = F.relu(self.l_in(self.e(c2)))
        in3 = F.relu(self.l_in(self.e(c3)))
        
        h = V(torch.zeros(in1.size()))
        h = F.tanh(self.l_hidden(h+in1))
        h = F.tanh(self.l_hidden(h+in2))
        h = F.tanh(self.l_hidden(h+in3))
        
        return F.log_softmax(self.l_out(h))

In [26]:
md = ColumnarModelData.from_arrays('.', [-1], np.stack([x1,x2,x3], axis=1), y, bs=512)

In [27]:
m = Char3Model(vocab_size, n_fac)

In [28]:
it = iter(md.trn_dl)
*xs,yt = next(it)
t = m(*V(xs))

In [29]:
opt = optim.Adam(m.parameters(), 1e-2)

In [30]:
fit(m, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss                            
    0      2.911144   3.086874  



[array([3.08687])]

In [193]:
set_lrs(opt, 0.001)

In [194]:
fit(m, md, 1, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.84525  6.52312]                                 



### Test model

In [31]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [32]:
get_next('y. ')

'p'

In [33]:
get_next('ppl')

'e'

In [34]:
get_next(' th')

'e'

In [35]:
get_next('and')

' '

## Our first RNN!

### Create inputs

This is the size of our unrolled RNN.

In [36]:
cs=8

For each of 0 through 7, create a list of every 8th character with that starting point. These will be the 8 inputs to our model.

In [37]:
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(len(idx)-cs)]

Then create a list of the next character in each of these series. This will be the labels for our model.

In [38]:
c_out_dat = [idx[j+cs] for j in range(len(idx)-cs)]

In [39]:
xs = np.stack(c_in_dat, axis=0)

In [40]:
xs.shape

(39992, 8)

In [41]:
y = np.stack(c_out_dat)

So each column below is one series of 8 characters from the text.

In [42]:
xs[:cs,:cs]

array([[39, 41, 28, 29, 24, 26, 28,  1],
       [41, 28, 29, 24, 26, 28,  1,  1],
       [28, 29, 24, 26, 28,  1,  1,  1],
       [29, 24, 26, 28,  1,  1,  1, 42],
       [24, 26, 28,  1,  1,  1, 42, 44],
       [26, 28,  1,  1,  1, 42, 44, 39],
       [28,  1,  1,  1, 42, 44, 39, 39],
       [ 1,  1,  1, 42, 44, 39, 39, 38]])

...and this is the next character after each sequence.

In [43]:
y[:cs]

array([ 1,  1, 42, 44, 39, 39, 38, 42])

### Create and train model

In [44]:
val_idx = get_cv_idxs(len(idx)-cs-1)

In [45]:
md = ColumnarModelData.from_arrays('.', val_idx, xs, y, bs=512)

In [48]:
class CharLoopModel(nn.Module):
    # This is an RNN!
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.l_in = nn.Linear(n_fac, n_hidden)
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, n_hidden))
        for c in cs:
            inp = F.relu(self.l_in(self.e(c)))
            h = F.tanh(self.l_hidden(h+inp))
        
        return F.log_softmax(self.l_out(h), dim=-1)

In [49]:
m = CharLoopModel(vocab_size, n_fac)
opt = optim.Adam(m.parameters(), 1e-2)

In [50]:
fit(m, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss                            
    0      2.540714   2.313722  



[array([2.31372])]

In [84]:
set_lrs(opt, 0.001)

In [85]:
fit(m, md, 1, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.73588  1.75103]                                 



In [53]:
class CharLoopConcatModel(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.l_in = nn.Linear(n_fac+n_hidden, n_hidden)
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, n_hidden))
        for c in cs:
            inp = torch.cat((h, self.e(c)), 1)
            inp = F.relu(self.l_in(inp))
            h = F.tanh(self.l_hidden(inp))
        
        return F.log_softmax(self.l_out(h), dim=-1)

In [54]:
m = CharLoopConcatModel(vocab_size, n_fac)
opt = optim.Adam(m.parameters(), 1e-3)

In [55]:
it = iter(md.trn_dl)
*xs,yt = next(it)
t = m(*V(xs))

In [56]:
fit(m, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss                            
    0      2.955463   2.684819  



[array([2.68482])]

In [96]:
set_lrs(opt, 1e-4)

In [97]:
fit(m, md, 1, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.69008  1.69936]                                 



### Test model

In [57]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [58]:
get_next('for thos')

' '

In [59]:
get_next('part of ')

't'

In [60]:
get_next('queens a')

'n'

## RNN with pytorch

In [61]:
class CharRnn(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(1, bs, n_hidden))
        inp = self.e(torch.stack(cs))
        outp,h = self.rnn(inp, h)
        
        return F.log_softmax(self.l_out(outp[-1]), dim=-1)

In [64]:
m = CharRnn(vocab_size, n_fac)
opt = optim.Adam(m.parameters(), 1e-3)

In [65]:
it = iter(md.trn_dl)
*xs,yt = next(it)

In [66]:
t = m.e(V(torch.stack(xs)))
t.size()

torch.Size([8, 512, 42])

In [67]:
ht = V(torch.zeros(1, 512,n_hidden))
outp, hn = m.rnn(t, ht)
outp.size(), hn.size()

(torch.Size([8, 512, 256]), torch.Size([1, 512, 256]))

In [68]:
t = m(*V(xs)); t.size()

torch.Size([512, 76])

In [114]:
fit(m, md, 4, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.86065  1.84255]                                 
[ 1.       1.68014  1.67387]                                 
[ 2.       1.58828  1.59169]                                 
[ 3.       1.52989  1.54942]                                 



In [115]:
set_lrs(opt, 1e-4)

In [69]:
fit(m, md, 2, opt, F.nll_loss)

epoch      trn_loss   val_loss                            
    0      2.92928    2.668219  
    1      2.611671   2.442238                            



[array([2.44224])]

### Test model

In [70]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [71]:
get_next('for thos')

' '

In [72]:
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [73]:
get_next_n('for thos', 40)

'for thos the the the the the the the the the the'

## Multi-output model

### Setup

Let's take non-overlapping sets of characters this time

In [74]:
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(0, len(idx)-cs-1, cs)]

Then create the exact same thing, offset by 1, as our labels

In [75]:
c_out_dat = [[idx[i+j] for i in range(cs)] for j in range(1, len(idx)-cs, cs)]

In [76]:
xs = np.stack(c_in_dat)
xs.shape

(4999, 8)

In [77]:
ys = np.stack(c_out_dat)
ys.shape

(4999, 8)

In [78]:
xs[:cs,:cs]

array([[39, 41, 28, 29, 24, 26, 28,  1],
       [ 1,  1, 42, 44, 39, 39, 38, 42],
       [32, 37, 30,  2, 69, 57, 50, 69],
       [ 2, 43, 67, 70, 69, 57,  2, 58],
       [68,  2, 50,  2, 72, 64, 62, 50],
       [63,  9,  9, 72, 57, 50, 69,  2],
       [69, 57, 54, 63, 23,  2, 32, 68],
       [ 2, 69, 57, 54, 67, 54,  2, 63]])

In [79]:
ys[:cs,:cs]

array([[41, 28, 29, 24, 26, 28,  1,  1],
       [ 1, 42, 44, 39, 39, 38, 42, 32],
       [37, 30,  2, 69, 57, 50, 69,  2],
       [43, 67, 70, 69, 57,  2, 58, 68],
       [ 2, 50,  2, 72, 64, 62, 50, 63],
       [ 9,  9, 72, 57, 50, 69,  2, 69],
       [57, 54, 63, 23,  2, 32, 68,  2],
       [69, 57, 54, 67, 54,  2, 63, 64]])

### Create and train model

In [80]:
val_idx = get_cv_idxs(len(xs)-cs-1)

In [81]:
md = ColumnarModelData.from_arrays('.', val_idx, xs, ys, bs=512)

In [82]:
class CharSeqRnn(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(1, bs, n_hidden))
        inp = self.e(torch.stack(cs))
        outp,h = self.rnn(inp, h)
        return F.log_softmax(self.l_out(outp), dim=-1)

In [85]:
m = CharSeqRnn(vocab_size, n_fac)
opt = optim.Adam(m.parameters(), 1e-3)

In [86]:
it = iter(md.trn_dl)
*xst,yt = next(it)

In [87]:
def nll_loss_seq(inp, targ):
    sl,bs,nh = inp.size()
    targ = targ.transpose(0,1).contiguous().view(-1)
    return F.nll_loss(inp.view(-1,nh), targ)

In [137]:
fit(m, md, 4, opt, nll_loss_seq)

A Jupyter Widget

[ 0.       2.59241  2.40251]                                
[ 1.       2.28474  2.19859]                                
[ 2.       2.13883  2.08836]                                
[ 3.       2.04892  2.01564]                                



In [88]:
set_lrs(opt, 1e-4)

In [89]:
fit(m, md, 1, opt, nll_loss_seq)

epoch      trn_loss   val_loss                          
    0      4.32723    4.291134  



[array([4.29113])]

### Identity init!

In [92]:
m = CharSeqRnn(vocab_size, n_fac)
opt = optim.Adam(m.parameters(), 1e-2)

In [93]:
m.rnn.weight_hh_l0.data.copy_(torch.eye(n_hidden))


    1     0     0  ...      0     0     0
    0     1     0  ...      0     0     0
    0     0     1  ...      0     0     0
       ...          ⋱          ...       
    0     0     0  ...      1     0     0
    0     0     0  ...      0     1     0
    0     0     0  ...      0     0     1
[torch.FloatTensor of size 256x256]

In [94]:
fit(m, md, 4, opt, nll_loss_seq)

epoch      trn_loss   val_loss                          
    0      3.490897   3.1025    
    1      3.235987   2.869533                          
    2      3.063832   2.690163                          
    3      2.929215   2.572571                          



[array([2.57257])]

In [95]:
set_lrs(opt, 1e-3)

In [144]:
fit(m, md, 4, opt, nll_loss_seq)

A Jupyter Widget

[ 0.       1.84035  1.85742]                                
[ 1.       1.82896  1.84887]                                
[ 2.       1.81879  1.84281]                               
[ 3.       1.81337  1.83801]                                



## Stateful model

### Setup

In [96]:
from torchtext import vocab, data

from fastai.nlp import *
from fastai.lm_rnn import *

PATH='data/nietzsche/'

TRN_PATH = 'trn/'
VAL_PATH = 'val/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

%ls {PATH}

nietzsche.txt       nietzschesmall.txt  trn/                val/


In [97]:
%ls {PATH}trn

nietzschesmall.txt


In [98]:
TEXT = data.Field(lower=True, tokenize=list)
bs=64; bptt=8; n_fac=42; n_hidden=256

FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=3)

len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(75, 45, 1, 39383)

### RNN

In [99]:
class CharSeqStatefulRnn(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        self.vocab_size = vocab_size
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))

In [101]:
m = CharSeqStatefulRnn(md.nt, n_fac, 512)
opt = optim.Adam(m.parameters(), 1e-3)

In [23]:
fit(m, md, 4, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.81983  1.81247]                                 
[ 1.       1.63097  1.66228]                                 
[ 2.       1.54433  1.57824]                                 
[ 3.       1.48563  1.54505]                                 



In [102]:
set_lrs(opt, 1e-4)

fit(m, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss                            
    0      3.223129   2.929459  



[array([2.92946])]

### RNN loop

In [103]:
# From the pytorch source

def RNNCell(input, hidden, w_ih, w_hh, b_ih, b_hh):
    return F.tanh(F.linear(input, w_ih, b_ih) + F.linear(hidden, w_hh, b_hh))

In [104]:
class CharSeqStatefulRnn2(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        super().__init__()
        self.vocab_size = vocab_size
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNNCell(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        outp = []
        o = self.h
        for c in cs: 
            o = self.rnn(self.e(c), o)
            outp.append(o)
        outp = self.l_out(torch.stack(outp))
        self.h = repackage_var(o)
        return F.log_softmax(outp, dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))

In [107]:
m = CharSeqStatefulRnn2(md.nt, n_fac, 512)
opt = optim.Adam(m.parameters(), 1e-3)

In [108]:
fit(m, md, 4, opt, F.nll_loss)

epoch      trn_loss   val_loss                            
    0      2.634824   2.435599  
    1      2.397607   2.288882                            
    2      2.261155   2.169498                            
    3      2.158415   2.082467                            



[array([2.08247])]

### GRU

In [109]:
class CharSeqStatefulGRU(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        super().__init__()
        self.vocab_size = vocab_size
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.GRU(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))

In [110]:
# From the pytorch source code - for reference

def GRUCell(input, hidden, w_ih, w_hh, b_ih, b_hh):
    gi = F.linear(input, w_ih, b_ih)
    gh = F.linear(hidden, w_hh, b_hh)
    i_r, i_i, i_n = gi.chunk(3, 1)
    h_r, h_i, h_n = gh.chunk(3, 1)

    resetgate = F.sigmoid(i_r + h_r)
    inputgate = F.sigmoid(i_i + h_i)
    newgate = F.tanh(i_n + resetgate * h_n)
    return newgate + inputgate * (hidden - newgate)

In [112]:
m = CharSeqStatefulGRU(md.nt, n_fac, 512)

opt = optim.Adam(m.parameters(), 1e-3)

In [113]:
fit(m, md, 6, opt, F.nll_loss)

epoch      trn_loss   val_loss                            
    0      2.667542   2.425069  
    1      2.371198   2.23098                             
    2      2.207084   2.096106                            
    3      2.081467   1.982581                            
    4      1.971633   1.877548                            
    5      1.877026   1.794282                            



[array([1.79428])]

In [30]:
set_lrs(opt, 1e-4)

In [31]:
fit(m, md, 3, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.22708  1.36926]                                 
[ 1.       1.21948  1.3696 ]                                 
[ 2.       1.22541  1.36969]                                 



### Putting it all together: LSTM

In [114]:
from fastai import sgdr

n_hidden=512

In [115]:
class CharSeqStatefulLSTM(nn.Module):
    def __init__(self, vocab_size, n_fac, bs, nl):
        super().__init__()
        self.vocab_size,self.nl = vocab_size,nl
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.LSTM(n_fac, n_hidden, nl, dropout=0.5)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h[0].size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs):
        self.h = (V(torch.zeros(self.nl, bs, n_hidden)),
                  V(torch.zeros(self.nl, bs, n_hidden)))

In [123]:
m = CharSeqStatefulLSTM(md.nt, n_fac, 512, 2)
lo = LayerOptimizer(optim.Adam, m, 1e-2, 1e-5)

In [119]:
os.makedirs(f'{PATH}models', exist_ok=True)

In [120]:
fit(m, md, 2, lo.opt, F.nll_loss)

epoch      trn_loss   val_loss                            
    0      2.519296   2.222408  
    1      2.186267   1.941648                            



[array([1.94165])]

In [125]:
on_end = lambda sched, cycle: save_model(m, f'{PATH}models/cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
fit(m, md, 1, lo.opt, F.nll_loss, callbacks=cb)

epoch      trn_loss   val_loss                            
    0      1.616982   1.337493  


[array([1.33749])]

In [44]:
on_end = lambda sched, cycle: save_model(m, f'{PATH}models/cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
fit(m, md, 2**6-1, lo.opt, F.nll_loss, callbacks=cb)

A Jupyter Widget

[ 0.       1.46053  1.43462]                                 
[ 1.       1.51537  1.47747]                                 
[ 2.       1.39208  1.38293]                                 
[ 3.       1.53056  1.49371]                                 
[ 4.       1.46812  1.43389]                                 
[ 5.       1.37624  1.37523]                                 
[ 6.       1.3173   1.34022]                                 
[ 7.       1.51783  1.47554]                                 
[ 8.       1.4921   1.45785]                                 
[ 9.       1.44843  1.42215]                                 
[ 10.        1.40948   1.40858]                              
[ 11.        1.37098   1.36648]                              
[ 12.        1.32255   1.33842]                              
[ 13.        1.28243   1.31106]                              
[ 14.        1.25031   1.2918 ]                              
[ 15.        1.49236   1.45316]                              
[ 16.   

### Test

In [126]:
def get_next(inp):
    idxs = TEXT.numericalize(inp,device=-1)
    p = m(VV(idxs.transpose(0,1)))
    r = torch.multinomial(p[-1].exp(), 1)
    return TEXT.vocab.itos[to_np(r)[0]]

In [127]:
get_next('for thos')

'a'

In [128]:
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [129]:
print(get_next_n('for thos', 400))

for thosal hunscholarigines--ors antion  of that the undressent--and asturebpessomethertogether; and wills. got mo all,watherein perhaps-begaprediced than it will to uppersion icceptameths of the truented term stylless belief your, the subtle<unk> a do long amatiour one could words to cognize of the philosophy, his fain bus-doorg as lesapertion in in shhope conscial or "invermancertay they awayge part of his 
